# Model for Titanic ML

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv(r"C:\\Users\User\Documents\Uni Berlin\Module, Vorlesungen, Seminare - Texte, Folien etc\2. Sommersemester 2020\_Collab_statistical_learning\train.csv") #, index_col="PassengerId")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Stratified Sampling

In [3]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data['Pclass']): 
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index] # weirdly it doesn't seem to work

    # some workaround??
    # now it does work with NaNs? I didn't even change the code at all

### Doch nicht. Normal Sampling instead

In [4]:
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)
train_set_c = train_set.copy()
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
331,332,0,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S
733,734,0,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S
382,383,0,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S
704,705,0,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S
813,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S


## Transformations

extremely useful because you can just put the new data through the pipeline and preprocessing is done

In [5]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='most_frequent')
embarked_imp = imp.fit_transform(train_set[['Embarked']])
train_set['Embarked_imp'] = embarked_imp
train_set

#But how do I integrate the ndarray to my df???

<ipython-input-5-4b230b403c73>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['Embarked_imp'] = embarked_imp


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Embarked_imp
331,332,0,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S,S
733,734,0,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S,S
382,383,0,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S,S
704,705,0,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S,S
813,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,107,1,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,NaN,S,S
270,271,0,1,"Cairns, Mr. Alexander",male,NaN,0,0,113798,31.0000,NaN,S,S
860,861,0,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,NaN,S,S
435,436,1,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,B96 B98,S,S


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# from sklearn_pandas import CategoricalImputer

#First we create the pipeline for numerical variables
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

#Now we create the full pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

num_att = ['Age', 'SibSp', 'Parch', 'Fare']
cat_att = ['Sex'] #,'Embarked_imp'] for reasons I don't understand this changes shape to 712, 10
ord_att = ['Pclass']


full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_att),
    # ('cat_imp', SimpleImputer(missing_values = np.nan, strategy='most_frequent'), cat_att),
    ('cat', OneHotEncoder(), cat_att),
    ('ord', OrdinalEncoder(), ord_att)
])

prepd_data = full_pipeline.fit_transform(train_set)

In [7]:
prepd_data

array([[ 1.25364106, -0.47072241, -0.47934164, ...,  0.        ,
         1.        ,  0.        ],
       [-0.47728355, -0.47072241, -0.47934164, ...,  0.        ,
         1.        ,  1.        ],
       [ 0.21508629, -0.47072241, -0.47934164, ...,  0.        ,
         1.        ,  2.        ],
       ...,
       [ 0.90745614,  1.23056874, -0.47934164, ...,  0.        ,
         1.        ,  2.        ],
       [-1.1696534 ,  0.37992316,  2.04874166, ...,  1.        ,
         0.        ,  0.        ],
       [-0.63114352, -0.47072241,  0.78470001, ...,  0.        ,
         1.        ,  0.        ]])

In [8]:
prepd_data.shape

(712, 7)

In [9]:
prepd_df = pd.DataFrame(prepd_data, columns=['Age', 'SibSp', 'Parch', 'Fare', 'Survived', 'Sex', 'Pclass']) #, 'Embarked'])
prepd_df.head()

,Age,SibSp,Parch,Fare,Survived,Sex,Pclass
0,1.253641,-0.470722,-0.479342,-0.078684,0.0,1.0,0.0
1,-0.477284,-0.470722,-0.479342,-0.377145,0.0,1.0,1.0
2,0.215086,-0.470722,-0.479342,-0.474867,0.0,1.0,2.0
3,-0.246494,0.379923,-0.479342,-0.476230,0.0,1.0,2.0
4,-1.785093,2.931860,2.048742,-0.025249,1.0,0.0,2.0


## Model

In [10]:
from sklearn.ensemble import RandomForestClassifier

dec_t_model = RandomForestClassifier()
dec_t_model.fit(prepd_data, train_set['Survived']) # or rather prepd_df as X?

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
# some initial score
from sklearn.metrics import mean_squared_error

predictions = dec_t_model.predict(prepd_data)
labels = train_set['Survived'].copy()

rmse = np.sqrt(mean_squared_error(labels, predictions))
rmse

0.14514618744678442

## cross-validate

The proprocessed training set (prepd_data) is split into 5 subsets, each subset is predicted by the other 4 subsets and we calculate how accurate the predictions were (as a percentage of correct classifications).

In [12]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dec_t_model, prepd_data, train_set['Survived'],
                        scoring = 'accuracy', cv = 5)

# Follow this link to check out other scoring methods you can use with cross validation
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [13]:
print(' Scores: ', scores, '\n Mean of scores: ', scores.mean(), '\n Standard Deviation of scores: ', scores.std())

 Scores:  [0.82517483 0.8041958  0.78873239 0.79577465 0.80985915] 
 Mean of scores:  0.8047473653107456 
 Standard Deviation of scores:  0.012495636691138245


looks quite good, extremely stable score!

In [14]:
# cross validate with rmse

scores_rmse = cross_val_score(dec_t_model, prepd_data, train_set['Survived'],
                        scoring = 'neg_mean_squared_error', cv = 5)
tree_rmse_scores = np.sqrt(-scores_rmse)

print(' Scores: ', tree_rmse_scores, '\n Mean of scores: ', tree_rmse_scores.mean(), '\n Standard Deviation of scores: ', tree_rmse_scores.std())

 Scores:  [0.40104758 0.45802861 0.46723641 0.45963856 0.45963856] 
 Mean of scores:  0.4491179434613574 
 Standard Deviation of scores:  0.024247908598930733


looks less good? So let's fine tune

## Fine Tuning

In [15]:
from sklearn.model_selection import GridSearchCV

param_grid =[
    {'n_estimators': [3, 10, 30, 50, 100], 'max_features': [2, 4, 6]},
    {'bootstrap':[False], 'n_estimators': [3, 10, 30], 'max_features': [2, 3, 4]},
]

forrest_class = RandomForestClassifier()

grid_search = GridSearchCV(forrest_class, param_grid, cv=5,
                          scoring='accuracy', return_train_score = True)

grid_search.fit(prepd_data, train_set['Survived'])

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [16]:
grid_search.best_params_

{'max_features': 2, 'n_estimators': 30}

In [17]:
results = grid_search.cv_results_
for x, y in zip(results['mean_test_score'], results['params']):
    print(x, y)

0.794937456909288 {'max_features': 2, 'n_estimators': 3}
0.7991726583275879 {'max_features': 2, 'n_estimators': 10}
0.8075938146360683 {'max_features': 2, 'n_estimators': 30}
0.794937456909288 {'max_features': 2, 'n_estimators': 50}
0.8005712597261894 {'max_features': 2, 'n_estimators': 100}
0.7781246922091992 {'max_features': 4, 'n_estimators': 3}
0.8019304639022948 {'max_features': 4, 'n_estimators': 10}
0.8005712597261893 {'max_features': 4, 'n_estimators': 30}
0.8061853639318428 {'max_features': 4, 'n_estimators': 50}
0.8061755146262188 {'max_features': 4, 'n_estimators': 100}
0.7879542992219049 {'max_features': 6, 'n_estimators': 3}
0.7977445090121147 {'max_features': 6, 'n_estimators': 10}
0.7991234117994681 {'max_features': 6, 'n_estimators': 30}
0.8033980104402639 {'max_features': 6, 'n_estimators': 50}
0.7977543583177387 {'max_features': 6, 'n_estimators': 100}
0.780921895006402 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
0.792159952723333 {'bootstrap': False, '

In [18]:
best_model = RandomForestClassifier(n_estimators = 10)

best_model.fit(prepd_data, train_set['Survived'])
new_predictions = best_model.predict(prepd_data)

new_rmse = np.sqrt(mean_squared_error(labels, new_predictions))
new_rmse
# (test_set.Survived==predictions).sum()/len(test_set) # doesn't work for me: ValueError: Lengths must match to compare



0.20866079134014764

weird, seems to be worse. Was the first run very lucky?

## Final steps towards submission

In [19]:
#Gather data
full_train = data.copy()
submission_test_set = pd.read_csv(r"C:\\Users\User\Documents\Uni Berlin\Module, Vorlesungen, Seminare - Texte, Folien etc\2. Sommersemester 2020\_Collab_statistical_learning\test.csv")

#Pipeline transform both sets
full_train_prepd = full_pipeline.fit_transform(full_train)
submission_set_prepd = full_pipeline.fit_transform(submission_test_set)

#Fit model
best_model.fit(full_train_prepd, full_train.Survived)
preds = best_model.predict(submission_set_prepd)


In [20]:
#Put the stuff together in a Data Frame

submissions = pd.DataFrame({'PassengerId': submission_test_set.PassengerId,
                             'Survived': preds})
submissions

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [21]:
submissions.to_csv('1st_submission.csv', index=False)